In [5]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
!pip install ultralytics opencv-python pyzbar


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 30.3 MB/s eta 0:00:00


In [8]:
DATA_DIR = "/content/drive/MyDrive/QR_datasets"  # put your train/test folders here
TRAIN_DIR = f"{DATA_DIR}/train/train_images"
TEST_DIR = f"{DATA_DIR}/test/test_images"
OUTPUT_DIR = "/content/drive/MyDrive/multiqr_outputs"

import os
os.makedirs(OUTPUT_DIR, exist_ok=True)


In [6]:
import cv2, os
from pyzbar.pyzbar import decode as pyzbar_decode

def convert_bbox(size, box):
    W, H = size
    x_min, y_min, x_max, y_max = box
    x_c = (x_min + x_max) / 2.0 / W
    y_c = (y_min + y_max) / 2.0 / H
    w = (x_max - x_min) / W
    h = (y_max - y_min) / H
    return (x_c, y_c, w, h)

YOLO_IMG_DIR = f"{DATA_DIR}/datasets/qr/images"
YOLO_LABEL_DIR = f"{DATA_DIR}/datasets/qr/labels"
os.makedirs(YOLO_IMG_DIR, exist_ok=True)
os.makedirs(YOLO_LABEL_DIR, exist_ok=True)

for img_name in os.listdir(TRAIN_DIR):
    img_path = os.path.join(TRAIN_DIR, img_name)
    img = cv2.imread(img_path)
    if img is None: continue
    h, w = img.shape[:2]
    boxes = []

    # Pyzbar
    for obj in pyzbar_decode(img):
        x, y, bw, bh = obj.rect
        if bw > 30 and bh > 30:
            boxes.append([x, y, x+bw, y+bh])

    # OpenCV QRCodeDetector
    qr_detector = cv2.QRCodeDetector()
    _, _, points, _ = qr_detector.detectAndDecodeMulti(img)
    if points is not None:
        for p in points.astype(int):
            x_min, y_min = p[:,0].min(), p[:,1].min()
            x_max, y_max = p[:,0].max(), p[:,1].max()
            if (x_max-x_min) > 30 and (y_max-y_min) > 30:
                boxes.append([x_min, y_min, x_max, y_max])

    # Save YOLO labels
    if boxes:
        txt_path = os.path.join(YOLO_LABEL_DIR, img_name.replace(".jpg", ".txt"))
        with open(txt_path, "w") as f:
            for (x1,y1,x2,y2) in boxes:
                x_c,y_c,w_norm,h_norm = convert_bbox((w,h), [x1,y1,x2,y2])
                f.write(f"0 {x_c} {y_c} {w_norm} {h_norm}\n")
        cv2.imwrite(os.path.join(YOLO_IMG_DIR, img_name), img)


The error indicates that the `pyzbar` library requires an additional dependency that needs to be installed at the system level. We can install this using `apt-get`.

In [3]:
!apt-get update && apt-get install -y libzbar0

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 https://cli.github.com/packages stable/main amd64 Packages [347 B]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:7 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,809 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,327 kB]
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:13 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 P

After running the cell above, the `pyzbar` library should be able to find the necessary shared library and the code should run without the `ImportError`.

In [14]:
yaml_path = f"{DATA_DIR}/datasets/qr/data.yaml"
with open(yaml_path, "w") as f:
    f.write(f"""
train: {YOLO_IMG_DIR}
val: {YOLO_IMG_DIR}  # same split if no val set
nc: 1
names: ["qr"]
""")


In [6]:
from ultralytics import YOLO

model = YOLO("yolov8m.pt")
model.train(
    data="/content/drive/MyDrive/QR_datasets/datasets/qr/data.yaml",
    epochs=100,
    imgsz=960,
    batch=8,
    model="yolov8m.pt",
    name="qr_detector_aug",
    augment=True ,   # ✅ correct spelling
    val=False
)


Ultralytics 8.3.204 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/QR_datasets/datasets/qr/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=960, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=qr_detector_aug2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=10

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7d6ccd6c6180>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [9]:
import json
results = model.predict(TEST_DIR, conf=0.25, save=True)

output = []
for r in results:
    img_id = os.path.basename(r.path)
    qrs = []
    for box in r.boxes.xyxy.cpu().numpy():
        x1,y1,x2,y2 = map(int, box)
        qrs.append({"bbox":[x1,y1,x2,y2]})
    output.append({"image_id": img_id, "qrs": qrs})

with open(f"{OUTPUT_DIR}/submission_detection_1.json", "w") as f:
    json.dump(output, f, indent=2)

print("✅ submission_detection_1.json saved")



image 1/50 /content/drive/MyDrive/QR_datasets/test/test_images/img201.jpg: 960x544 1 qr, 158.8ms
image 2/50 /content/drive/MyDrive/QR_datasets/test/test_images/img202.jpg: 544x960 1 qr, 138.9ms
image 3/50 /content/drive/MyDrive/QR_datasets/test/test_images/img203.jpg: 960x544 7 qrs, 111.0ms
image 4/50 /content/drive/MyDrive/QR_datasets/test/test_images/img204.jpg: 960x544 5 qrs, 95.1ms
image 5/50 /content/drive/MyDrive/QR_datasets/test/test_images/img205.jpg: 960x544 5 qrs, 110.7ms
image 6/50 /content/drive/MyDrive/QR_datasets/test/test_images/img206.jpg: 960x544 4 qrs, 87.3ms
image 7/50 /content/drive/MyDrive/QR_datasets/test/test_images/img207.jpg: 960x544 5 qrs, 111.7ms
image 8/50 /content/drive/MyDrive/QR_datasets/test/test_images/img208.jpg: 960x544 4 qrs, 87.7ms
image 9/50 /content/drive/MyDrive/QR_datasets/test/test_images/img209.jpg: 960x544 6 qrs, 83.6ms
image 10/50 /content/drive/MyDrive/QR_datasets/test/test_images/img210.jpg: 960x544 7 qrs, 110.9ms
image 11/50 /content/dri

In [11]:
import cv2, os
def classify_qr(value):
    if value.startswith("B"): return "batch"
    elif value.startswith("MFR"): return "manufacturer"
    else: return "other"

bonus_output = []
for r in results:
    img = cv2.imread(r.path)
    img_id = os.path.basename(r.path)
    qrs = []
    for box in r.boxes.xyxy.cpu().numpy():
        x1,y1,x2,y2 = map(int, box)
        crop = img[y1:y2, x1:x2]
        val, _, _ = cv2.QRCodeDetector().detectAndDecode(crop)
        if val:
            qrs.append({"bbox":[x1,y1,x2,y2], "value":val, "type":classify_qr(val)})
    bonus_output.append({"image_id": img_id, "qrs": qrs})

with open(f"{OUTPUT_DIR}/submission_decoding_2.json", "w") as f:
    json.dump(bonus_output, f, indent=2)

print("✅ submission_decoding_2.json saved")


✅ submission_decoding_2.json saved


In [21]:
# Validate on val set
metrics = model.val()
print(metrics)


Ultralytics 8.3.204 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
val: Fast image access ✅ (ping: 0.5±0.3 ms, read: 254.8±209.2 MB/s, size: 1906.0 KB)
val: Scanning /content/drive/MyDrive/QR_datasets/datasets/qr/labels.cache... 196 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 196/196 280.2Kit/s 0.0s
train: /content/drive/MyDrive/QR_datasets/datasets/qr/images/img140.jpg: 1 duplicate labels removed
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 25/25 1.3it/s 19.4s
                   all        196        995      0.986      0.749      0.814      0.807
Speed: 2.1ms preprocess, 76.1ms inference, 0.0ms loss, 2.1ms postprocess per image
Results saved to /content/runs/detect/qr_detector_aug23
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7d6d60699670